In [7]:
import cv2
import numpy as np
import os
from glob import glob


In [8]:
# โฟลเดอร์ input/output
input_dir = "./data/prepared-2"       # เปลี่ยนเป็น path ของโฟลเดอร์รูป
output_dir = "./data/cropped"  # เปลี่ยนเป็น path ของโฟลเดอร์ผลลัพธ์
os.makedirs(output_dir, exist_ok=True)

In [9]:
# pip install opencv-python pandas
import cv2, numpy as np, os, pandas as pd
from glob import glob

# ---------- helpers ----------
def calibrate_ppcm(bgr):
    """หา px/cm จากสี่เหลี่ยมแดง 3×3 cm; return (ppcm, red_rect)"""
    hsv = cv2.cvtColor(bgr, cv2.COLOR_BGR2HSV)
    r1 = cv2.inRange(hsv, (0,80,80), (10,255,255))
    r2 = cv2.inRange(hsv, (170,80,80), (180,255,255))
    red = cv2.morphologyEx(r1 | r2, cv2.MORPH_CLOSE, np.ones((5,5),np.uint8), 2)
    cnts,_ = cv2.findContours(red, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not cnts: return None, None
    cnts = sorted(cnts, key=lambda c: cv2.contourArea(c), reverse=True)[:5]
    best, err_best = None, 1e9
    for c in cnts:
        x,y,w,h = cv2.boundingRect(c)
        if w*h < 60: continue
        err = abs(w/h - 1.0)
        if err < err_best:
            err_best, best = err, (x,y,w,h)
    if not best: return None, None
    x,y,w,h = best
    ppcm = ((w+h)/2.0)/3.0   # px / cm
    return ppcm, best

def preprocess_and_segment(bgr):
    """ลบ marker สี + ลดกริด แล้วคืน binary mask ของวัตถุมืด (ผลไม้)"""
    # ลดนอยส์+คมภาพ
    bgr = cv2.bilateralFilter(bgr, d=9, sigmaColor=40, sigmaSpace=40)
    bgr = cv2.addWeighted(bgr, 1.6, cv2.GaussianBlur(bgr,(9,9),0), -0.6, 0)

    hsv = cv2.cvtColor(bgr, cv2.COLOR_BGR2HSV)
    # marker สี (ตัดออก)
    blue = cv2.inRange(hsv, (90,70,60), (130,255,255))
    r1   = cv2.inRange(hsv, (0,80,80),  (10,255,255))
    r2   = cv2.inRange(hsv, (170,80,80),(180,255,255))
    markers = cv2.morphologyEx(np.uint8(((blue>0)|(r1>0)|(r2>0))*255), cv2.MORPH_DILATE, np.ones((7,7),np.uint8), 1)

    # วัตถุเข้ม (ผลไม้) บน V channel
    V = hsv[:,:,2]
    base = cv2.inRange(V, 0, 120)

    # ทำความสะอาด
    mask = cv2.bitwise_and(base, cv2.bitwise_not(markers))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN,  np.ones((5,5),np.uint8), 1)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((11,11),np.uint8), 2)
    return mask

def pick_best_contour(mask_roi, x_offset=0, area_min=300):
    """เลือก contour ที่เป็นผลไม้ใน ROI ตามเกณฑ์พื้นที่+ความกลม; ส่งคืน contour (พิกัด global)"""
    cnts,_ = cv2.findContours(mask_roi, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    best, score_best = None, -1
    for c in cnts:
        area = cv2.contourArea(c)
        if area < area_min:
            continue
        peri = cv2.arcLength(c, True)
        circ = 4*np.pi*area/(peri*peri + 1e-8)  # ใกล้ 1 = กลม
        x,y,w,h = cv2.boundingRect(c)
        ar = w/float(h) if h>0 else 0
        # ให้คะแนน: พื้นที่ใหญ่ + กลม + อัตราส่วนใกล้ 1
        score = area + 800*circ - 50*abs(ar-1.0)
        if score > score_best:
            score_best, best = score, c
    if best is None:
        return None
    # shift พิกัด ROI → global
    best = best.copy()
    best[:,:,0] += x_offset
    return best

def measure_all(contour, ppcm):
    xs = contour[:,:,0].astype(np.int32).flatten()
    ys = contour[:,:,1].astype(np.int32).flatten()
    # 1) กว้าง/สูงจาก extreme points (ผลไม้ล้วน)
    W_cm = (xs.max()-xs.min())/ppcm
    H_cm = (ys.max()-ys.min())/ppcm
    # 2) วงรีฟิต
    MA_cm = ma_cm = np.nan; angle = np.nan
    if len(contour) >= 5:
        (cx,cy),(MA,ma),angle = cv2.fitEllipse(contour)
        MA_cm, ma_cm = MA/ppcm, ma/ppcm
    # 3) วงกลมล้อมรอบ
    cc, r = cv2.minEnclosingCircle(contour)
    D_cm = (2*r)/ppcm
    return dict(W_cm=W_cm, H_cm=H_cm, MA_cm=MA_cm, ma_cm=ma_cm, D_cm=D_cm, angle_deg=angle)

def annotate(img, ppcm, red_rect, c_left, m_left, c_right, m_right):
    vis = img.copy()
    font = cv2.FONT_HERSHEY_SIMPLEX
    if red_rect:
        x,y,w,h = red_rect
        cv2.rectangle(vis, (x,y), (x+w,y+h), (0,0,255), 2)
    cv2.putText(vis, f"ppcm: {ppcm:.2f} px/cm", (8,20), font, 0.55, (20,20,20), 2, cv2.LINE_AA)
    cv2.putText(vis, f"ppcm: {ppcm:.2f} px/cm", (8,20), font, 0.55, (240,240,240), 1, cv2.LINE_AA)

    # วาดซ้าย (#1) – ข้อความด้านบน
    if c_left is not None:
        cv2.drawContours(vis, [c_left], -1, (0,255,0), 2)
        if len(c_left) >= 5:
            cv2.ellipse(vis, cv2.fitEllipse(c_left), (255,255,0), 2)
        cc, r = cv2.minEnclosingCircle(c_left)
        center_x, center_y = int(cc[0]), int(cc[1])
        cv2.circle(vis, (center_x, center_y), int(r), (255,0,255), 2)
        x1,y1,w1,h1 = cv2.boundingRect(c_left)
        text1 = f"#1 W={m_left['W_cm']:.2f}  H={m_left['H_cm']:.2f}  MA={m_left['MA_cm']:.2f}  ma={m_left['ma_cm']:.2f}  D={m_left['D_cm']:.2f} cm"
        ty = max(24, y1-10)                   # ด้านบน
        cv2.putText(vis, text1, (x1, ty), font, 0.3, (30,30,30), 2, cv2.LINE_AA)
        cv2.putText(vis, text1, (x1, ty), font, 0.3, (240,240,240), 1, cv2.LINE_AA)

    # วาดขวา (#2) – ข้อความด้านซ้าย
    if c_right is not None:
        cv2.drawContours(vis, [c_right], -1, (0,255,0), 2)
        if len(c_right) >= 5:
            cv2.ellipse(vis, cv2.fitEllipse(c_right), (255,255,0), 2)
        cc, r = cv2.minEnclosingCircle(c_right)
        center_x, center_y = int(cc[0]), int(cc[1])
        cv2.circle(vis, (center_x, center_y), int(r), (255,0,255), 2)
        x2,y2,w2,h2 = cv2.boundingRect(c_right)
        text2 = f"#2 W={m_right['W_cm']:.2f}  H={m_right['H_cm']:.2f}  MA={m_right['MA_cm']:.2f}  ma={m_right['ma_cm']:.2f}  D={m_right['D_cm']:.2f} cm"
        tx = max(8, x2-130)                    # ด้านซ้าย
        ty = y2 + h2 + 20                     # กึ่งกลางแนวตั้ง
        cv2.putText(vis, text2, (tx, ty), font, 0.3, (30,30,30), 2, cv2.LINE_AA)
        cv2.putText(vis, text2, (tx, ty), font, 0.3, (240,240,240), 1, cv2.LINE_AA)

    return vis

# ---------- main loop ----------
rows = []
image_files = []
for ext in ("*.jpg","*.png","*.jpeg","*.bmp"):
    image_files += glob(os.path.join(input_dir, ext))
image_files.sort()

for fp in image_files:
    img = cv2.imread(fp);
    if img is None:
        print("skip:", fp);
        continue

    ppcm, red_rect = calibrate_ppcm(img)
    if not ppcm:
        print("No red square → skip:", fp)
        continue

    mask = preprocess_and_segment(img)
    H, W = mask.shape

    # แบ่งซ้าย/ขวาเป็น ROI
    left_roi  = mask[:, :W//2]
    right_roi = mask[:, W//2:]

    c_left  = pick_best_contour(left_roi,  x_offset=0,     area_min=350)
    c_right = pick_best_contour(right_roi, x_offset=W//2,  area_min=350)

    if c_left is None or c_right is None:
        print("Could not find both fruits in:", fp)
        continue

    m_left  = measure_all(c_left,  ppcm)
    m_right = measure_all(c_right, ppcm)

    # annotate: #1 ด้านซ้าย (ข้อความบน), #2 ด้านขวา (ข้อความล่าง)
    vis = annotate(img, ppcm, red_rect, c_left, m_left, c_right, m_right)

    # save image + rows
    out_img = os.path.join(output_dir, os.path.splitext(os.path.basename(fp))[0] + "_measured.jpg")
    cv2.imwrite(out_img, vis)

    rows.append({"image": os.path.basename(fp), "side": "left(#1)",  **m_left,  "ppcm_px_per_cm": ppcm})
    rows.append({"image": os.path.basename(fp), "side": "right(#2)", **m_right, "ppcm_px_per_cm": ppcm})

# export CSV
df = pd.DataFrame(rows)
csv_path = os.path.join(output_dir, "measurements_multi.csv")
df.to_csv(csv_path, index=False)
print("Done. Images at:", output_dir)
print("CSV:", csv_path)


Done. Images at: ./data/cropped
CSV: ./data/cropped/measurements_multi.csv
